In [9]:
import torch
import numpy as np
import matplotlib.pyplot as plt
# importing packages
from PIL import Image
import glob
import sklearn.metrics as metrics
import warnings
warnings.filterwarnings("ignore")
import copy 

embedding = torch.load("Embeddings Generated/unseen_embeddings_train_L2.pth")
km = torch.load("Cluster Centroid Embeddings/10_allmean_level2 Kmeans_88x2048_resnet_k_8.pth")


cluster_centre = {}
exp = 1
mul = 1
embedding_list = []
embedding_map = {}
for i, unseen_class in enumerate(embedding.keys()):
    embedding_list.append(np.array(embedding[unseen_class][0].flatten().cpu()))
    embedding_map[unseen_class] = i
    
embedding_arr = np.array(torch.tensor(embedding_list))
km_cluster_centers = km.cluster_centers_

from sklearn.preprocessing import StandardScaler,MinMaxScaler
scaler = StandardScaler()
embedding_arr = scaler.fit_transform(embedding_arr)
km_cluster_centers = scaler.transform(km_cluster_centers)

emedding_old = copy.deepcopy(embedding)
for i, unseen_class in enumerate(emedding_old.keys()):
    embedding[unseen_class] = embedding_arr[embedding_map[unseen_class]]
    
for i, clust in enumerate(km_cluster_centers):
    cluster_centre[i] = torch.unsqueeze(
        torch.unsqueeze(torch.tensor(clust), dim=1), dim=2)

# In[]:
    
assigned_clust = {}
for unseen_class in embedding.keys():
    unseen_class_embedding = embedding[unseen_class]
    assigned_clust[unseen_class] = 0
    maxSim = -1
    point2 = torch.pow(torch.unsqueeze(torch.tensor(unseen_class_embedding.flatten()), dim=0).cpu() * mul, exp)
    for clust_key in cluster_centre.keys():
        point1 = torch.pow(torch.unsqueeze(torch.tensor(cluster_centre[clust_key]).flatten(), dim=0).cpu() * mul, exp)        
        sim = metrics.pairwise.cosine_distances(point1, point2)
        if sim[0][0] > maxSim:
            assigned_clust[unseen_class] = str(clust_key)
            maxSim = sim[0][0]

print(assigned_clust)

{'american_paint_horse': '1', 'sparrow': '5', 'rhinoceros': '1', 'ragdoll_cat': '4', 'tuna': '0', 'catfish': '4', 'giraffe': '4', 'crow': '5', 'burmese_cat': '5', 'british_shorthair_cat': '6', 'puffbird': '6', 'ibis': '1', 'cuckoo': '0', 'parrot': '2', 'camarillo_white_horse': '0', 'kingfisher': '5', 'friesian_horse': '1', 'american_bobtail': '5', 'kiwi_bird': '5', 'deer': '5'}
